The objective of the project is to train a model that will result in a mathematical equation. This equation will use specific input features (such as temperature, water quality, sunlight exposure, etc.) to predict the percentage of coral bleaching. The goal is for this equation to serve as a practical tool, enabling people to pinpoint factors contributing to coral bleaching and to guide them in taking steps to mitigate or prevent it.

### DATA PREPROCESSING

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer  
from sklearn.preprocessing import OneHotEncoder      
from sklearn.model_selection import KFold   
from statistics import mean
import joblib


In [93]:
raw_data = pd.read_csv(r'.\dataset\global_bleaching_environmental_cleaned.csv')

In [94]:
print('\n____________ Dataset info ____________')
print(raw_data.info())       


____________ Dataset info ____________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41184 entries, 0 to 41183
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Latitude_Degrees     41184 non-null  float64
 1   Longitude_Degrees    41184 non-null  float64
 2   Ocean_Name           41184 non-null  object 
 3   Realm_Name           41184 non-null  object 
 4   Ecoregion_Name       41181 non-null  object 
 5   Distance_to_Shore    41182 non-null  float64
 6   Exposure             41184 non-null  object 
 7   Turbidity            41178 non-null  float64
 8   Cyclone_Frequency    41184 non-null  float64
 9   Depth_m              39387 non-null  float64
 10  Percent_Cover        28764 non-null  float64
 11  Bleaching_Level      41184 non-null  object 
 12  Percent_Bleaching    34407 non-null  float64
 13  ClimSST              41073 non-null  float64
 14  Temperature_Kelvin   41038 non-null  float64
 

handle the outlier using iqr quantile


### Handle the missing value

#### Calculate the percentage of missing value in each column

In [95]:
# Calculate the number of missing values per column
missing_values_count = raw_data.isnull().sum()

# Calculate the percentage of missing values per column
total_rows = len(raw_data)
missing_percentage = (missing_values_count / total_rows) * 100

# Create a DataFrame to display the results
missing_data_df = pd.DataFrame({
    'Missing Values': missing_values_count,
    'Percentage': missing_percentage
})

print(missing_data_df)

                     Missing Values  Percentage
Latitude_Degrees                  0    0.000000
Longitude_Degrees                 0    0.000000
Ocean_Name                        0    0.000000
Realm_Name                        0    0.000000
Ecoregion_Name                    3    0.007284
Distance_to_Shore                 2    0.004856
Exposure                          0    0.000000
Turbidity                         6    0.014569
Cyclone_Frequency                 0    0.000000
Depth_m                        1797    4.363345
Percent_Cover                 12420   30.157343
Bleaching_Level                   0    0.000000
Percent_Bleaching              6777   16.455420
ClimSST                         111    0.269522
Temperature_Kelvin              146    0.354507
Temperature_Mean                130    0.315657
Temperature_Maximum             130    0.315657
Windspeed                       127    0.308372
SSTA                            146    0.354507
SSTA_Mean                       130    0

#### Remove the column that have the percentage of missing > 25% (Percent_cover);

In [96]:
column_to_remove = 'Percent_Cover'
# Remove the column
raw_data = raw_data.drop(columns=[column_to_remove])

##### Reason for this:
1. Bias Reduction:
High Missing Rate: If a column has a high percentage of missing values, any attempt to fill in those missing values (e.g., through imputation) can introduce significant bias. The imputed values may not accurately represent the true data, leading to unreliable models.

Reduced Data Quality: Columns with a lot of missing data can degrade the quality of your dataset, as imputed values might not capture the variability and true relationships within the data.

2. Simplification of the Model:
Avoiding Overfitting: Including columns with many missing values might increase the complexity of the model, leading to overfitting. Removing such columns can simplify the model, making it more generalizable.

Improving Interpretability: Fewer, more relevant features make it easier to interpret and understand the model. Columns with high missing values often contribute little to the model's predictive power.

3. Efficient Use of Resources:
Reduced Computational Load: By removing columns with a high percentage of missing values, you reduce the dimensionality of your dataset, leading to faster training and testing times. This is particularly important when working with large datasets.

#### Handling the categorical_columns


In [97]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Load the data
raw_data = pd.read_csv(r'.\dataset\global_bleaching_environmental_cleaned.csv')

# Categorical columns
categorical_columns = ['Ecoregion_Name']

# Impute missing values with the most frequent value (mode)
imputer_categorical = SimpleImputer(strategy='most_frequent')
raw_data[categorical_columns] = imputer_categorical.fit_transform(raw_data[categorical_columns])

# Display the result to ensure missing values are imputed
print(raw_data[categorical_columns].isnull().sum())


Ecoregion_Name    0
dtype: int64


#### Handling for numerical and continuous features


In [98]:
# Numerical columns
numerical_columns = ['Depth_m', 'Percent_Bleaching', 'ClimSST', 'Temperature_Kelvin', 
                     'Temperature_Mean', 'Temperature_Maximum', 'Windspeed', 'SSTA', 
                     'SSTA_Mean', 'SSTA_Maximum', 'SSTA_Frequency', 'SSTA_DHW', 
                     'TSA', 'TSA_Maximum', 'TSA_Mean', 'TSA_Frequency', 'TSA_DHW']

# Impute missing values with the median (or change strategy to 'mean' if appropriate)
imputer_numerical = SimpleImputer(strategy='median')
raw_data[numerical_columns] = imputer_numerical.fit_transform(raw_data[numerical_columns])

# Display the result to ensure missing values are imputed
print(raw_data[numerical_columns].isnull().sum())

Depth_m                0
Percent_Bleaching      0
ClimSST                0
Temperature_Kelvin     0
Temperature_Mean       0
Temperature_Maximum    0
Windspeed              0
SSTA                   0
SSTA_Mean              0
SSTA_Maximum           0
SSTA_Frequency         0
SSTA_DHW               0
TSA                    0
TSA_Maximum            0
TSA_Mean               0
TSA_Frequency          0
TSA_DHW                0
dtype: int64


### Handle the outliers

#### The way to handle the outlier

Handling outliers using the Interquartile Range (IQR) is a common and effective method. The IQR is the range between the first quartile (Q1) and the third quartile (Q3) and represents the middle 50% of the data. Outliers are typically defined as any data points that fall below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR.

Steps to Handle Outliers Using IQR:
Calculate the IQR: Compute Q1 and Q3 for each numerical feature.
Identify Outliers: Determine the lower and upper bounds using the IQR.
Handle Outliers: Depending on the situation, you can:
Remove Outliers: Drop the rows containing outliers.
Cap Outliers: Replace outliers with the nearest value within the acceptable range (often called winsorizing).
Transform Outliers: Apply a transformation (e.g., log) to reduce the impact of outliers.

In [99]:
import pandas as pd

# Load the data
raw_data = pd.read_csv(r'.\dataset\global_bleaching_environmental_cleaned.csv')

# List of actual numerical columns
numerical_columns = [
    'Latitude_Degrees', 'Longitude_Degrees', 'Distance_to_Shore', 'Exposure', 
    'Turbidity', 'Cyclone_Frequency', 'Depth_m', 'Percent_Cover', 'Bleaching_Level', 
    'Percent_Bleaching', 'ClimSST', 'Temperature_Kelvin', 'Temperature_Mean', 
    'Temperature_Maximum', 'Windspeed', 'SSTA', 'SSTA_Mean', 'SSTA_Maximum', 
    'SSTA_Frequency', 'SSTA_DHW', 'TSA', 'TSA_Maximum', 'TSA_Mean', 
    'TSA_Frequency', 'TSA_DHW'
]

# Function to calculate IQR and handle outliers
def handle_outliers_iqr(df, columns):
    for col in columns:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[col]):
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Option 1: Remove outliers
            # df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
            
            # Option 2: Cap outliers
            df[col] = df[col].apply(lambda x: upper_bound if x > upper_bound else lower_bound if x < lower_bound else x)
        else:
            print(f"Skipping non-numeric column: {col}")
        
    return df

# Apply the function to handle outliers
raw_data = handle_outliers_iqr(raw_data, numerical_columns)

# Display the result to ensure outliers are handled
print(raw_data.describe())


Skipping non-numeric column: Exposure
Skipping non-numeric column: Bleaching_Level
       Latitude_Degrees  Longitude_Degrees  Distance_to_Shore     Turbidity  \
count      41184.000000       41184.000000       41182.000000  41178.000000   
mean           7.561469          34.958047        1260.287545      0.060503   
std           15.732738         103.414234        1547.845199      0.041034   
min          -30.262500        -179.974300           3.200000      0.000000   
25%           -4.715150         -78.397800         124.762500      0.033500   
50%           10.771300          96.843300         457.610000      0.052300   
75%           20.051000         120.880400        1789.150000      0.079400   
max           36.750000         179.964500        4285.731250      0.148250   

       Cyclone_Frequency       Depth_m  Percent_Cover  Percent_Bleaching  \
count       41184.000000  39387.000000   28764.000000       34407.000000   
mean           51.963563      6.881721      19.469663